# Yöntem 1: **CBAM Channel + Coordinate Attention** (Tek Spatial Mekanizma)

Bu defter, **CBAM’den sadece kanal dikkatini (Channel Attention)** alıp, **uzamsal/konumsal tarafı Coordinate Attention ile** çözmeyi anlatır.  
Amaç: **iki farklı spatial gate’i üst üste bindirmeden** (over‑suppression riskini artırmadan) hem kanal seçiciliği hem de eksen‑tabanlı konum duyarlılığı elde etmek.

---

## Neden “Yöntem 1”?
CBAM’in klasik yapısı: **CA (channel) + SA (spatial)**.  
Coordinate Attention ise **(H ve W eksenlerinden) konum bilgisi taşıyan** bir tür spatial gating üretir.

Bu nedenle:
- CBAM’in **SA** kısmını da açıp üstüne Coordinate eklemek → çoğu senaryoda *maskelerin çarpılması* gibi davranır ve **aşırı bastırma** yapabilir.
- Daha stabil tasarım: **CBAM’den CA al → Coordinate uygula → residual ile karıştır**.

Bu defterde tam olarak bu tasarım uygulanır:

\[ x \rightarrow \text{ChannelAttentionFusionT} \rightarrow \text{CoordinateAttPlus} \rightarrow \text{Residual Mix} \]


## Blokların kısa özeti

### 1) ChannelAttentionFusionT (CBAM‑CA’nın geliştirilmiş hali)
- Girişten **global ortalama** ve **global maksimum** özet çıkarır.
- Aynı MLP (1×1 conv) üzerinden iki özet işlenir.
- **Fusion (sum veya softmax router)** ile `avg` ve `max` katkıları birleştirilir.
- Sonuç **temperature (T)** ile ölçeklenir ve **sigmoid / hardsigmoid** ile kanal maskesi üretilir.

Önemli detaylar:
- `fusion="softmax"` seçilirse router son katmanı **sıfır init** yapılır → eğitim başında **avg/max dengeli** başlar.
- `temperature=0.9` gibi değerler maskeyi biraz daha keskinleştirir (çok yüksek T → fazla yumuşak, çok düşük T → çok sert).

### 2) CoordinateAttPlus (H/W eksen profili + multi‑scale DW conv)
- `H_profile`: W boyunca özet (mean + max karışımı) → şekil `(B,C,H,1)`
- `W_profile`: H boyunca özet (mean + max karışımı) → şekil `(B,C,1,W)`
- Her eksen için:
  - **local depthwise** (k=3)
  - **dilated depthwise** (k=3, dilation=d)
  - sonra **1×1 channel mixer**
- H ve W özellikleri birleştirilir, ortak bottleneck’ten geçer, tekrar split edilir.
- `attn_h` ve `attn_w` ile eksen‑maskeleri üretilir ve çarpılıp **beta** ile yumuşatılır.

Önemli detaylar:
- `mid` (bottleneck genişliği) küçük kalırsa temsil gücü düşer.  
  Bu yüzden `mid_floor = max(8, min(32, C//4))` gibi adaptif taban kullanılır.
- `beta` spatial ölçeklemeyi yumuşatır: `scale = 1 + beta*(scale-1)`
- `alpha_h/alpha_w` eksen maskesinin gücünü öğrenilebilir şekilde ayarlar (0→no‑effect, 1→tam mask).

### 3) Residual Mix
Son çıktı:
- residual açıkken: `out = x + alpha*(y - x)`
- residual kapalıysa: `out = y`

Buradaki `alpha` da logit uzayında tutulur, `sigmoid(alpha_raw)` ile [0,1] aralığına alınır.


## Pratik yerleştirme rehberi (CNN / YOLO gibi yapılarda)

Bu blok genelde şu noktalarda iyi çalışır:
- **Backbone orta katmanları**: Özellikle C=64/128/256 gibi kanalların arttığı bölgeler.
- **Neck (FPN/PAN)**: Birleştirme sonrası gürültüyü azaltıp kanal seçiciliği kazandırabilir.
- **Head’e çok yakın** yerlerde: daha agresif olabilir, dikkatli ayarlanmalı.

Öneri:
- İlk denemede **tek bir seviyede** ekle (ör. backbone’un bir stage’i).
- Sonra katman sayısını artır veya neck’e taşı.
- Her eklemede: **std_ratio**, **mask stats**, **latency** kontrol et.

İlk ayar seti (stabil başlangıç):
- CA: `fusion="softmax"`, `temperature=0.9`, `learnable_temperature=True` (opsiyonel)
- Coord: `beta=0.35`, `dilation=2`, `norm="gn"`
- Residual: `alpha_init=0.75`, `learnable_alpha=False` (başlangıçta sabit tutmak stabilite sağlar)


## Kod

Aşağıdaki kod; **ChannelAttentionFusionT**, **CoordinateAttPlus** ve wrapper **CBAMChannelPlusCoord** sınıflarını içerir.  
Kod içinde sadece kritik yerlerde kısa açıklama vardır; asıl mantık açıklaması markdown hücrelerinde verilmiştir.


----
----

## Kodun işleyiş kısmına aşina olanlar içindir.Eğer tam açıklamaları incelemek isterseniz bu dosyayı ziyaret edin :: "Attention Mekanizmaları\CBAM + Cordinat Attention Block (5)\Yöntem -1\Kod_Anlatımları.ipynb"

**Bu yapıda amaç, feature map üzerinde bilgiyi iki aşamalı ve kontrollü şekilde yeniden ağırlıklandırmak. İlk aşamada kanal bazında, ikinci aşamada ise uzamsal (konumsal) bazda dikkat uygulanıyor.**

**İlk olarak CBAM’in kanal attention kısmı çalışıyor. Burada her kanal için global ortalama ve global maksimum değerler çıkarılıyor. Ortalama değer, kanalın tüm uzay boyunca genel aktivasyon seviyesini temsil ederken; maksimum değer o kanaldaki en baskın, en ayırt edici sinyali temsil ediyor. Bu iki farklı bakış açısı, kanalın gerçekten anlamlı olup olmadığını değerlendirmek için birlikte kullanılıyor.**

*Bu kanal özetleri küçük bir öğrenilebilir yapıdan geçirilerek her kanal için bir attention değeri üretiliyor. Ortalama ve maksimum bilgilerin katkısı sabit tutulmuyor; model, bu iki bilginin hangisinin daha önemli olduğunu öğrenebiliyor. Temperature parametresi burada devreye girerek attention maskesinin daha sert mi yoksa daha yumuşak mı uygulanacağını ayarlıyor. Böylece kanal attention, veri setine ve öğrenme sürecine göre kendini adapte edebiliyor*.*

**Kanal attention tamamlandıktan sonra Coordinate Attention aşamasına geçiliyor. Bu aşamada artık soru “hangi kanal önemli” değil, “bu önemli kanal görüntünün hangi bölgesinde etkili” sorusu oluyor. Bunun için yükseklik (H) ve genişlik (W) yönleri ayrı ayrı ele alınıyor. Feature map, H ve W eksenlerinde özetlenerek iki farklı uzamsal profil oluşturuluyor. Bu özetleme işlemi hem ortalama hem de maksimum değerler kullanılarak yapılıyor; böylece hem genel uzamsal dağılım hem de lokal olarak baskın bölgeler korunmuş oluyor.**

*Bu uzamsal profiller, hem lokal hem de daha geniş bağlamı yakalayabilmek için farklı uzamsal işlemlerden geçiriliyor. Sonrasında bu bilgiler birleştirilerek her kanal için H ve W yönlü uzamsal attention maskeleri üretiliyor. Bu maskeler, kanal attention’dan geçen feature map’e uygulanarak uzamsal olarak da seçici bir yapı elde ediliyor.*

**Coordinate Attention’da attention’ın gücü doğrudan sabitlenmiyor. Öğrenilebilir katsayılar sayesinde, uzamsal attention’ın ne kadar baskın olacağı ayarlanabiliyor. Böylece model, bazı durumlarda uzamsal dikkat mekanizmasını güçlü şekilde kullanırken, bazı durumlarda daha geri planda tutabiliyor**

**Son aşamada ise bu iki attention aşamasından geçen çıktı, residual bir yapı ile orijinal feature map ile birleştiriliyor. Bu birleşim de kontrollü bir şekilde yapılıyor. Yani model isterse attention etkisini azaltabiliyor, isterse tamamen kullanabiliyor. Bu yaklaşım, özellikle YOLO gibi hassas mimarilerde attention’ın eğitimi bozmasını veya gereğinden fazla bastırma yapmasını engelliyor.**

* Özetle bu blok, kanal bazlı önemlendirme ile uzamsal konum bilgisini birlikte kullanarak, bilgiyi daha seçici, daha dengeli ve daha stabil bir şekilde öne çıkaran bir attention mekanizması sunuyor.

---

In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F


def _softplus_inverse(y: torch.Tensor, eps: float = 1e-6) -> torch.Tensor:
    return torch.log(torch.clamp(torch.exp(y) - 1.0, min=eps))


def _get_gate(gate: str):
    g = gate.lower()
    if g == "sigmoid":
        return torch.sigmoid
    if g == "hardsigmoid":
        return F.hardsigmoid
    raise ValueError("gate 'sigmoid' veya 'hardsigmoid' olmalı.")


def _get_act(act: str):
    a = act.lower()
    if a == "relu":
        return nn.ReLU(inplace=True)
    if a == "silu":
        return nn.SiLU(inplace=True)
    raise ValueError("act 'relu' veya 'silu' olmalı.")


class ChannelAttentionFusionT(nn.Module):
    def __init__(
        self,
        channels: int,
        reduction: int = 16,
        min_hidden: int = 4,
        fusion: str = "softmax",
        gate: str = "sigmoid",
        temperature: float = 0.9,
        learnable_temperature: bool = False,
        eps: float = 1e-6,
        act: str = "relu",
        bias: bool = True,
        fusion_router_hidden: int = 16,
        return_fusion_weights: bool = False,
    ):
        super().__init__()
        if channels < 1:
            raise ValueError("channels >= 1 olmalı.")
        if reduction < 1:
            raise ValueError("reduction >= 1 olmalı.")
        if fusion not in ("sum", "softmax"):
            raise ValueError("fusion 'sum' veya 'softmax' olmalı.")
        if temperature <= 0:
            raise ValueError("temperature pozitif olmalı.")
        if fusion_router_hidden < 1:
            raise ValueError("fusion_router_hidden >= 1 olmalı.")

        self.eps = float(eps)
        self.fusion = fusion
        self.return_fusion_weights = bool(return_fusion_weights)
        self.gate_fn = _get_gate(gate)

        hidden = max(int(min_hidden), int(channels) // int(reduction))
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.fc1 = nn.Conv2d(channels, hidden, kernel_size=1, bias=bias)
        self.act = _get_act(act)
        self.fc2 = nn.Conv2d(hidden, channels, kernel_size=1, bias=bias)

        if self.fusion == "softmax":
            self.fusion_router = nn.Sequential(
                nn.Conv2d(2 * channels, fusion_router_hidden, kernel_size=1, bias=True),
                nn.ReLU(inplace=True),
                nn.Conv2d(fusion_router_hidden, 2, kernel_size=1, bias=True),
            )

            last = self.fusion_router[-1] # Sequential’ın son katmanını yakalama
            nn.init.zeros_(last.weight)
            nn.init.zeros_(last.bias)
            # self.fusion_router bir nn.Sequential. [-1] demek “içindeki en son layer”. 
            # Burada en son layer Conv2d(..., out_channels=2) olan katman.
            # Bunu alıp last.weight ve last.bias’ı sıfırlıyoruz → başlangıçta router tarafsız başlasın diye.
            # İlk adımda logits = 0 olur → softmax([0,0]) = [0.5, 0.5] → avg ve max eşit ağırlıklı başlar (stabil başlangıç).

        else:
            self.fusion_router = None

        self.learnable_temperature = bool(learnable_temperature)
        if self.learnable_temperature:
            t0 = torch.tensor(float(temperature))
            t_inv = _softplus_inverse(t0, eps=self.eps)
            self.t_raw = nn.Parameter(t_inv)
        else:
            self.register_buffer("T", torch.tensor(float(temperature)))

    def get_T(self) -> torch.Tensor:
        if self.learnable_temperature:
            return F.softplus(self.t_raw) + self.eps
        return self.T

    def mlp(self, s: torch.Tensor) -> torch.Tensor:
        return self.fc2(self.act(self.fc1(s)))

    def forward(self, x: torch.Tensor):
        avg_s = self.avg_pool(x)
        max_s = self.max_pool(x)

        a = self.mlp(avg_s)
        m = self.mlp(max_s)

        fusion_w = None
        if self.fusion == "sum":
            z = a + m
        else:
            s_cat = torch.cat([avg_s, max_s], dim=1)
            logits = self.fusion_router(s_cat).flatten(1)
            fusion_w = torch.softmax(logits, dim=1) 
            # fusion_w shape: (B, 2)
            # Her örnek için iki sayı: [w_avg, w_max].
            # a ve m shape: (B, C, 1, 1)
            # fusion_w[:, 0] shape (B,) olur. Bunu (B,1,1,1) yapıyoruz 
            # ki PyTorch broadcasting ile (B,C,1,1) ile çarpılabilsin.

            w0 = fusion_w[:, 0].view(-1, 1, 1, 1)
            w1 = fusion_w[:, 1].view(-1, 1, 1, 1)
            z = w0 * a + w1 * m # burada w0 her batch sample için “avg tarafının genel katsayısı”, w1 max tarafının katsayısı.

        T = self.get_T().to(device=x.device, dtype=x.dtype)
        ca = self.gate_fn(z / T)
        y = x * ca

        if self.return_fusion_weights and (fusion_w is not None):
            return y, ca, fusion_w
        return y, ca


class HSwish(nn.Module):
    def forward(self, x):
        return x * F.relu6(x + 3.0, inplace=True) / 6.0


def make_norm(norm: str, ch: int):
    norm = norm.lower()
    if norm == "bn":
        return nn.BatchNorm2d(ch)
    if norm == "gn":
        g = min(32, ch)
        while ch % g != 0 and g > 2:
            g //= 2
        if ch % g != 0:
            g = 2 if (ch % 2 == 0) else 1
        return nn.GroupNorm(g, ch)
    if norm == "none":
        return nn.Identity()
    raise ValueError("norm 'none', 'bn', 'gn' dışında olamaz.")


class CoordinateAttPlus(nn.Module):
    def __init__(
        self,
        in_channels: int,
        reduction: int = 32,
        min_mid_channels: int = 8,
        act: str = "hswish",
        init_alpha: float = 0.7,
        learnable_alpha: bool = True,
        beta: float = 0.35,
        dilation: int = 2,
        norm: str = "gn",
        use_spatial_gate: bool = False,
        spatial_gate_beta: float = 0.35,
    ):
        super().__init__()

        if in_channels < 1:
            raise ValueError("in_channels >= 1 olmalı.")
        if reduction < 1:
            raise ValueError("reduction >= 1 olmalı.")
        if dilation < 1:
            raise ValueError("dilation >= 1 olmalı.")

        # SEBEBİ :: Aşırı dar bottleneck’i engelleme (capacity tabanı)
        mid_floor = max(8, min(32, int(in_channels) // 4))
        # in_channels//reduction bazı kanallarda çok küçük olabilir (örn 64//32=2). 
        # Bu, koordinat bilgisini taşıyan bottleneck’i “nefessiz” bırakır.
        # mid_floor ile minimum kapasiteyi garanti ediyoruz :en az 8 ama çok da şişmesin diye 
        # 32’ye kadar sınırayrıca kanal büyüdükçe in_channels //4 ile ölçeklenebilir.
        mid = max(int(min_mid_channels), int(in_channels) // int(reduction))
        mid = max(mid, int(mid_floor))
        # CA/Coord gibi attention bloklarında bottleneck çok küçülürse 
        # “öğrenilecek” şey kalmıyor → maske ya düz/işe yaramaz oluyor ya da saçmalıyor. 
        # Bu kademeli seçim onu engelliyor.

        act_l = act.lower()
        if act_l == "hswish":
            self.act = HSwish()
        elif act_l == "relu":
            self.act = nn.ReLU(inplace=True)
        elif act_l == "silu":
            self.act = nn.SiLU(inplace=True)
        else:
            raise ValueError("act 'hswish', 'relu', 'silu' olmalı.")

        self.shared_bottleneck_proj = nn.Conv2d(in_channels, mid, 1, bias=False)
        self.shared_bottleneck_norm = make_norm(norm, mid)
        self.shared_bottleneck_refine = nn.Conv2d(mid, mid, 1, bias=False)
        self.shared_bottleneck_refine_norm = make_norm(norm, mid)

        self.h_local_dw = nn.Conv2d(
            in_channels, in_channels, kernel_size=(3, 1), padding=(1, 0), groups=in_channels, bias=False
        )
        self.w_local_dw = nn.Conv2d(
            in_channels, in_channels, kernel_size=(1, 3), padding=(0, 1), groups=in_channels, bias=False
        )

        d = int(dilation)
        self.h_dilated_dw = nn.Conv2d(
            in_channels,
            in_channels,
            kernel_size=(3, 1),
            padding=(d, 0),
            dilation=(d, 1),
            groups=in_channels,
            bias=False,
        )
        self.w_dilated_dw = nn.Conv2d(
            in_channels,
            in_channels,
            kernel_size=(1, 3),
            padding=(0, d),
            dilation=(1, d),
            groups=in_channels,
            bias=False,
        )

        self.h_channel_mixer = nn.Conv2d(in_channels, in_channels, 1, bias=True)
        self.w_channel_mixer = nn.Conv2d(in_channels, in_channels, 1, bias=True)

        self.h_attention_head = nn.Conv2d(mid, in_channels, 1, bias=True)
        self.w_attention_head = nn.Conv2d(mid, in_channels, 1, bias=True)

        self.beta = float(beta)

        # “alpha 0–1 aralığında kalsın” diye parametreyi logits uzayında tutma
        eps = 1e-6
        a0 = float(init_alpha)
        a0 = min(max(a0, eps), 1.0 - eps)
        raw0 = torch.logit(torch.tensor(a0), eps=eps) 
        # torch.logit(a0) sigmoid’in tersidir.
        # Böylece başlangıçta sigmoid(alpha_raw) = init_alpha olur.

        if learnable_alpha:
            self.alpha_h_raw = nn.Parameter(raw0.clone())
            self.alpha_w_raw = nn.Parameter(raw0.clone())
        else:
            self.register_buffer("alpha_h_raw", raw0.clone())
            self.register_buffer("alpha_w_raw", raw0.clone())

        self.use_spatial_gate = bool(use_spatial_gate)
        self.spatial_gate_beta = float(spatial_gate_beta)
        if self.use_spatial_gate:
            self.spatial_gate_dw = nn.Conv2d(in_channels, in_channels, 3, padding=1, groups=in_channels, bias=False)
            self.spatial_gate_pw = nn.Conv2d(in_channels, in_channels, 1, bias=True)

        self._last_ah = None
        self._last_aw = None

    def forward(self, x: torch.Tensor):
        _, _, H, W = x.shape

        # %50-%50 karışım → hem stabiliteyi hem “salient” sinyali taşır.
        h_profile = 0.5 * (x.mean(dim=3, keepdim=True) + x.amax(dim=3, keepdim=True))
        # mean: genel enerji/ortalama aktivasyon → stabil, gürültüye dayanıklı
        # max: sivri/aykırı güçlü aktivasyon → “önemli obje izi” gibi pikleri yakalar ama gürültüye hassas
        # Bu profile’lar sonra H ve W yönünde attention üretmek için “özet kanal” oluyor.
        w_profile = 0.5 * (x.mean(dim=2, keepdim=True) + x.amax(dim=2, keepdim=True))

        h_ms = self.h_channel_mixer(self.h_local_dw(h_profile) + self.h_dilated_dw(h_profile))
        w_ms = self.w_channel_mixer(self.w_local_dw(w_profile) + self.w_dilated_dw(w_profile))
        w_ms = w_ms.permute(0, 1, 3, 2)

        hw = torch.cat([h_ms, w_ms], dim=2)

        mid = self.act(self.shared_bottleneck_norm(self.shared_bottleneck_proj(hw)))
        mid = self.act(self.shared_bottleneck_refine_norm(self.shared_bottleneck_refine(mid)))

        mid_h, mid_w = torch.split(mid, [H, W], dim=2)
        mid_w = mid_w.permute(0, 1, 3, 2)

        attn_h = F.hardsigmoid(self.h_attention_head(mid_h), inplace=False)
        attn_w = F.hardsigmoid(self.w_attention_head(mid_w), inplace=False)

        self._last_ah = attn_h.detach()
        self._last_aw = attn_w.detach()

        alpha_h = torch.sigmoid(self.alpha_h_raw).to(device=x.device, dtype=x.dtype)
        alpha_w = torch.sigmoid(self.alpha_w_raw).to(device=x.device, dtype=x.dtype)

        scale_h = (1.0 - alpha_h) + alpha_h * attn_h
        scale_w = (1.0 - alpha_w) + alpha_w * attn_w

        scale = scale_h * scale_w
        scale = 1.0 + self.beta * (scale - 1.0)

        out = x * scale

        if self.use_spatial_gate:
            sg = self.spatial_gate_pw(self.spatial_gate_dw(x))
            sg = F.hardsigmoid(sg, inplace=False)
            sg = 1.0 + self.spatial_gate_beta * (sg - 1.0)
            out = out * sg

        return out

    @torch.no_grad()
    def last_mask_stats(self):
        if (self._last_ah is None) or (self._last_aw is None):
            return None
        ah = self._last_ah
        aw = self._last_aw
        return {
            "a_h": {"min": float(ah.min()), "mean": float(ah.mean()), "max": float(ah.max()), "std": float(ah.std())},
            "a_w": {"min": float(aw.min()), "mean": float(aw.mean()), "max": float(aw.max()), "std": float(aw.std())},
        }


class CBAMChannelPlusCoord(nn.Module):
    def __init__(
        self,
        channels: int,
        ca_reduction: int = 16,
        ca_min_hidden: int = 4,
        ca_fusion: str = "softmax",
        ca_gate: str = "sigmoid",
        ca_temperature: float = 0.9,
        ca_act: str = "relu",
        ca_fusion_router_hidden: int = 16,
        learnable_temperature: bool = False,
        coord_reduction: int = 32,
        coord_min_mid: int = 8,
        coord_act: str = "hswish",
        coord_init_alpha: float = 0.7,
        coord_learnable_alpha: bool = True,
        coord_beta: float = 0.35,
        coord_dilation: int = 2,
        coord_norm: str = "gn",
        coord_use_spatial_gate: bool = False,
        coord_spatial_gate_beta: float = 0.35,
        residual: bool = True,
        alpha_init: float = 0.75,
        learnable_alpha: bool = False,
        return_maps: bool = False,
    ):
        super().__init__()
        if channels < 1:
            raise ValueError("channels >= 1 olmalı.")

        self.return_maps = bool(return_maps)
        self.residual = bool(residual)

        self.ca = ChannelAttentionFusionT(
            channels=channels,
            reduction=ca_reduction,
            min_hidden=ca_min_hidden,
            fusion=ca_fusion,
            gate=ca_gate,
            temperature=ca_temperature,
            learnable_temperature=learnable_temperature,
            eps=1e-6,
            act=ca_act,
            bias=True,
            fusion_router_hidden=ca_fusion_router_hidden,
            return_fusion_weights=self.return_maps,
        )

        self.coord = CoordinateAttPlus(
            in_channels=channels,
            reduction=coord_reduction,
            min_mid_channels=coord_min_mid,
            act=coord_act,
            init_alpha=coord_init_alpha,
            learnable_alpha=coord_learnable_alpha,
            beta=coord_beta,
            dilation=coord_dilation,
            norm=coord_norm,
            use_spatial_gate=coord_use_spatial_gate,
            spatial_gate_beta=coord_spatial_gate_beta,
        )

        # : Residual karışım şiddetini kontrol eden tek skaler
        # Bu alpha blok çıktısının ne kadar uygulanacağını belirler.
        # alpha=0 → tamamen input (blok kapalı)
        # alpha=1 → tamamen blok çıktısıarası → yumuşak karışım
        # Aynı “0–1 aralığı garantisi” için alpha_raw logit uzayında tutuluyor.

        if self.residual:
            eps = 1e-6
            a0 = float(alpha_init)
            a0 = min(max(a0, eps), 1.0 - eps)
            raw0 = torch.logit(torch.tensor(a0), eps=eps)
            if learnable_alpha:
                self.alpha_raw = nn.Parameter(raw0)
            else:
                self.register_buffer("alpha_raw", raw0)

    # Cihaz/dtype uyumlu sabit üretme ve güvenli fallback
    def _alpha(self, x: torch.Tensor) -> torch.Tensor:
        # residual=True iken alpha_raw yoksa (normalde olmamalı) güvenli fallback: alpha=1.0
        if not hasattr(self, "alpha_raw"):
            # hasattr kontrolü → teorik olarak residual açıkken alpha_raw olmalı. 
            # Ama bir bug/yanlış init olursa crash yerine alpha=1.0 ile devam etsin diye güvenlik.
            return x.new_tensor(1.0) 
        # x.new_tensor(1.0) → x ile aynı device (CPU/GPU) ve aynı dtype ile 1.0 üretir.
        # Bu sayede “device mismatch” hatası olmaz.
        return torch.sigmoid(self.alpha_raw).to(device=x.device, dtype=x.dtype)

    # Pipeline ve debug çıktıları
    def forward(self, x: torch.Tensor):

        # y = self.ca(x) :: :: Kanal bazında (C dimension) ölçekleme yapar. Her kanalın “ne kadar açık kalacağını” belirler.
        # y = self.coord(y) :: :: Sonra koordinat tabanlı H/W attention uygular: yatay ve dikey eksen boyunca içerik + konum ilişkisini işler.
        # out = x + alpha*(y - x) (residual açık ise) :: Bu blok “tam bastırma” yapmasın diye yumuşatır. Alpha küçükse blok etkisi yumuşak olur.
        if self.return_maps:
        # return_maps=True iken ekstra debug verisi döndürür:
        # ca_map: (B,C,1,1)
        # fusion_w: (B,2) (softmax router ağırlıkları)
        # coord_stats: son H/W mask istatistikleri
        
            y, ca_map, fusion_w = self.ca(x)
            y = self.coord(y)

            if self.residual:
                out = x + self._alpha(x) * (y - x)
            else:
                out = y

            coord_stats = self.coord.last_mask_stats()
            return out, ca_map, fusion_w, coord_stats

        y, _ = self.ca(x)
        y = self.coord(y)

        if self.residual:
            out = x + self._alpha(x) * (y - x)
        else:
            out = y

        return out


## Hızlı sanity kontrolü

Aşağıdaki hücre; shape kontrolü ve temel istatistikleri basitçe gösterir:
- Çıkış shape’i girişle aynı mı?
- CA mask ortalama/min/max
- Coordinate H/W mask istatistikleri
- Router fusion ağırlıkları (softmax router açıkken)

Not: Bu sadece hızlı kontrol içindir; gerçek karar için küçük bir eğitim/validasyon koşusu ve ablation gerekir.


In [2]:

torch.manual_seed(0)

x = torch.randn(2, 64, 56, 56)

m = CBAMChannelPlusCoord(
    channels=64,
    return_maps=True,
    residual=True,
    alpha_init=0.75,
    learnable_alpha=False,
    learnable_temperature=True,
    ca_temperature=0.9,
    coord_beta=0.35,
    coord_dilation=2,
    coord_norm="gn",
)

out, ca_map, fusion_w, coord_stats = m(x)

print("x:", x.shape)
print("out:", out.shape)
print("ca_map:", ca_map.shape)
print("fusion_w:", fusion_w.shape)
print("coord_stats:", coord_stats)

print("CA stats:", float(ca_map.min()), float(ca_map.mean()), float(ca_map.max()), float(ca_map.std()))
print("fusion_w mean:", fusion_w.mean(dim=0).tolist())


x: torch.Size([2, 64, 56, 56])
out: torch.Size([2, 64, 56, 56])
ca_map: torch.Size([2, 64, 1, 1])
fusion_w: torch.Size([2, 2])
coord_stats: {'a_h': {'min': 0.0, 'mean': 0.4944933354854584, 'max': 0.9390336871147156, 'std': 0.053503669798374176}, 'a_w': {'min': 0.0, 'mean': 0.5012840628623962, 'max': 0.9842378497123718, 'std': 0.06626447290182114}}
CA stats: 0.31093117594718933 0.5261633992195129 0.7153375744819641 0.10749468952417374
fusion_w mean: [0.5, 0.5]


C:\Users\hdgn5\AppData\Local\Temp\ipykernel_18368\4011596079.py:26: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\autograd\generated\python_variable_methods.cpp:837.)
  print("CA stats:", float(ca_map.min()), float(ca_map.mean()), float(ca_map.max()), float(ca_map.std()))


## Eğitim sırasında gözlenmesi gereken 4 sinyal

1) **Çıkış/Giriş std oranı**
- `std_ratio = out.std() / x.std()` çok düşerse (ör. 0.2–0.3) → attention muhtemelen fazla bastırıyor.
- hedef band genelde kabaca 0.6–1.1 arası (modele göre değişir).

2) **CA mask dağılımı**
- mean ~ 0.5 civarı normal.
- min çok düşük (0.05 gibi) ve max çok düşükse → çok bastırma.

3) **Fusion weights**
- Router bir tarafa “kilitleniyorsa” (örn sürekli [0.99, 0.01]) → ya veri buna itiyor ya da T/MLP çok agresif.
- İlk epoch’larda dengeli olması normal; sonra hafif kayması normal.

4) **Coordinate H/W mask stats**
- `a_h` ve `a_w` mean/std çok küçük kalırsa → coordinate etkisi zayıf (mid düşük, beta çok düşük, alpha_h/w düşük).
- Çok keskinleşirse → beta veya dilation ayarı gözden geçirilmeli.

Bu sinyaller, attention’ı “güçlü mü, doğru mu, gereksiz mi?” diye ayıklamak için yeterli olur.


## Ablation planı (önerilen)

Aynı backbone/neck üzerinde sırayla:

- **Baseline:** attention yok
- **CA only:** sadece `ChannelAttentionFusionT`
- **Coord only:** sadece `CoordinateAttPlus`
- **Yöntem 1:** `CA -> Coord -> residual` (bu defterin konusu)

Her adımda:
- mAP / accuracy (hedef metriğin)
- latency / FPS
- param ve MAC
- std_ratio, mask stats

Eğer Yöntem 1 baseline’a göre net kazanıyorsa, daha ileri karmaşıklık (Yöntem 2: paralel SA gibi) ancak o zaman anlamlı olur.


# 🔎 Attention Bloğu Çıktıları – Özet Tablo

| Çıktı               | Shape                | Ne Anlama Geliyor                  | Senin Çıktında Ne Söylüyor                                    |
| ------------------- | -------------------- | ---------------------------------- | ------------------------------------------------------------- |
| **x**               | `[2, 64, 56, 56]`    | Giriş feature map                  | Backbone’dan gelen ham özellikler                             |
| **out**             | `[2, 64, 56, 56]`    | Attention + residual sonrası çıktı | Shape korunmuş → bilgi eklenmedi, sadece yeniden ağırlıklandı |
| **ca_map**          | `[2, 64, 1, 1]`      | Kanal attention maskesi            | Her kanal için tek önem katsayısı                             |
| **fusion_w**        | `[2, 2]`             | AvgPool–MaxPool ağırlıkları        | Avg ve Max eşit (0.5–0.5) → tarafsız başlangıç                |
| **coord_stats.a_h** | scalar istatistikler | Yükseklik (H) yönlü attention      | H ekseninde dengeli ama seçici odak                           |
| **coord_stats.a_w** | scalar istatistikler | Genişlik (W) yönlü attention       | W ekseninde benzer şekilde stabil                             |
| **CA stats**        | min/mean/max/std     | Kanal attention dağılımı           | Agresif değil, bastırma yok                                   |


# 📌 Detaylı Ama Kısa Yorumlar

| Ölçüm                   | Değer       | Yorumu                            |
| ----------------------- | ----------- | --------------------------------- |
| `ca_map.mean ≈ 0.53`    | Orta seviye | Kanallar hafif güçlendirilmiş     |
| `ca_map.std ≈ 0.11`     | Düşük       | Kanal attention stabil            |
| `fusion_w = [0.5, 0.5]` | Dengeli     | Avg / Max arasında taraf tutmuyor |
| `a_h.mean ≈ 0.49`       | Dengeli     | H ekseninde yumuşak attention     |
| `a_w.mean ≈ 0.50`       | Dengeli     | W ekseninde yumuşak attention     |
| `a_h.max ≈ 0.94`        | Yüksek      | Bazı bölgelerde net odak var      |
| `a_w.max ≈ 0.98`        | Yüksek      | Uzamsal seçicilik mevcut          |
| `std (H/W) ≈ 0.05–0.06` | Düşük       | Gürültüsüz spatial attention      |


# 1️⃣ Channel Attention (CA) tarafı
## ❌ Kötü durumlar ve anlamları
| Gözlenen değer           | Bu ne demek                 | Neden kötü             |
| ------------------------ | --------------------------- | ---------------------- |
| `ca_map.mean < 0.3`      | Kanallar aşırı bastırılıyor | Bilgi kaybı, mAP düşer |
| `ca_map.mean > 0.8`      | Her kanal önemli sanılıyor  | Attention etkisizleşir |
| `ca_map.std > 0.25`      | Kanal seçimi çok sert       | Eğitim kararsızlaşır   |
| `ca_map.min ≈ 0` çok sık | Kanallar ölüyor             | Gradient akışı bozulur |


### 🛠️ Nasıl düzeltilir?

Çok küçükse → temperature ↑ veya alpha ↓

Çok büyükse → temperature ↓ veya gate’i hardsigmoid → sigmoid

std çok yüksekse → residual alpha_init düşü

# 2️⃣ Fusion Weights (fusion_w)


## ❌ Kötü durumlar 
| Durum                              | Anlamı                   | Sorun                  |
| ---------------------------------- | ------------------------ | ---------------------- |
| `[1.0, 0.0]` sabit                 | Sadece avg kullanılıyor  | Max bilgisi boşa       |
| `[0.0, 1.0]` sabit                 | Sadece max kullanılıyor  | Global bağlam kaybolur |
| Erken aşamada uç değerlere gitmesi | Router erken kilitlenmiş | Genelleme bozulur      |


### 🛠️ Nasıl düzeltilir?

Router son katmanını zero-init

Router hidden çok büyükse küçült

Early training’de LR düşür

# 3️⃣ Coordinate Attention – H / W istatistikleri 
## ❌ Kötü durumlar

| Gözlem              | Anlamı                     | Sonuç                      |
| ------------------- | -------------------------- | -------------------------- |
| `mean < 0.3`        | Uzamsal bastırma çok fazla | Küçük objeler kaybolur     |
| `mean > 0.8`        | Her yer önemli             | Spatial attention anlamsız |
| `std > 0.15`        | Gürültülü spatial mask     | False positive artar       |
| `min = 0` her yerde | Sert spatial gate          | Detection head zarar görür |

### 🛠️ Nasıl düzeltilir?

Çok agresifse → beta ↓

Etkisizse → beta ↑

Gürültülüyse → dilated conv sayısını azalt

Small object kayboluyorsa → use_spatial_gate=False

# 4️⃣ Residual Alpha (alpha_raw)
## ❌ Kötü durumlar
| Durum                             | Anlamı                          |
| --------------------------------- | ------------------------------- |
| `alpha ≈ 1`                       | Attention her şeyi eziyor       |
| `alpha ≈ 0`                       | Attention boşa çalışıyor        |
| Learnable alpha hızlı 1’e gitmesi | Model attention’a aşırı bağımlı |

**Kötü attention; çok baskılayan, erken kilitlenen ve gürültü üreten attention’dırBizim çıktılarımız ise yumuşak, dengeli ve öğrenmeye açık bir attention gösteriyor.**